<a href="https://colab.research.google.com/github/parkerburchett/EconCapstone/blob/main/Get_All_Monthly_Income_from_Etherscan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Constants
parker_wallet = '0x76fb6d38f28c44a13380220df21363bd7af45ee1'
ethan_wallet = '0xceb4d0ca821420cf2553b9e244f6b52364613f94'

## This is the first draft of the notebook I am writing for my Economics Capstone to get a published paper. \

https://www.etherchain.org/charts Look at "mining revenue" gives USD value per day of a GH/s. Use this instead of building it yourself.
### Outline: What will this notebook do?

1. Use the Etherscan.io charts to convert into production 
* Block Number to Date.(Year, Month)
* (Year, Month) to Monthly average Value of 1 day of mining at 1 GH/s (Etherscan.io chart). Just average by month. 

* (Year, Month) to Average ETH Price. 


* You will need a dictionary that looks like 
(Year, Month): Average Gh/s needed to mine make 1 ETH 
### Called production dictionary. 

2. You will need to write custom Estimate_hashrate 

*Pseudocode:
  Estimate_HashRate(Year, Month, ETH earned this month):

  estimated_hashrate_in_ghS = Production_Dictionary[(Year, Month) * ETH earned  this month)


3. Use the Etherscan.io API to get all of the transactions from 5 large pools, and store that in pyspark on Google Colab. You will need to save that to someplace in the cloud so that you can get it later and put the public link. I think drop box is the right place for it.

 -> What I want the data to look like.
 This information is factual raw data. 
(Block Number of Transaction, Pool Address, Miner Address, Amount of ETH)


Every Single Transaction will be mapped into this form. 

(Block Number, Day, Year, Month, Pool Address, Miner Address, Amount of ETH, Estimated_hashrate)

This data will then be grouped by Miner_address and Month.

Intermediate(after group by) result:

Month, Year, Pool Address, Miner Address, ETH Earned this month, Estimated Hashrate, (ETH earned this month * Average Monthly Price of ETH) as monthly USD Revenue.

Frame work for Miner Features
This is how I am choosing to categorize Miners. 

## Miner Features
Start month ( cast this as an int with January 2015 as 1). # month of first  income).
End Month (cast this as an int with March 2021 as the 80 or whatever the number happens to be). Last month with income. or + infinity if that was march 2021. Age as number of full months,
Median Hashrate, as size. 



Dummy Varibles for Pool address, 
Dummy Variables for if they went to a different




Framework for OLS regression:

##Independent variable: 
* Estimated Monthly Hashrate.

###Dependent Variables
Miner Features. 
Year
Month,
Average ETH price,
average Gh/s value



### Setup Pyspark and Connect it to your google drive

In [12]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Open Connection to Google drive

from google.colab import drive
drive.mount('/content/drive')
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print("\n\nDRIVE SETUP")

openjdk-8-jdk-headless is already the newest version (8u282-b08-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


DRIVE SETUP


## Install Libraries 

In [13]:
import pandas as pd
import numpy as np
import json
import requests
import math
import datetime
import matplotlib.pyplot as plt
import time
%matplotlib inline

import pyspark
from functools import reduce
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import DataType
from pyspark import SparkContext, SparkConf


### Create Spark Session

In [14]:
# create the spark session
conf = SparkConf().set("spark.ui.port", "4050") # copy and pasted this, dont' know what it means
# create the context
# try catch block lets you start again from the top without causing problems. 
try:
  sc.stop()
  sc = pyspark.SparkContext(conf=conf)
except:
  sc = pyspark.SparkContext(conf=conf)
  
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .appName('my-cool-app') \
    .getOrCreate()


# you are just copy and pasting the spark config details

print('Successfully Created Spark Session')

Successfully Created Spark Session


## Methods to interact with the Etherscan.io API


In [15]:
def read_api_key():
  api_key_json = open(r'/content/drive/MyDrive/etherScan_apiKey.json') # read in my private Etherscan.io API key from my Google Drive
  etherscan_api_key =json.load(api_key_json)['key']
  api_key_json.close()
  return etherscan_api_key

ETHERSCAN_API_KEY = read_api_key() # You api key is a constant so the variable name is UPPER CASE
ETHERMINE_WALLET = '0xea674fdde714fd979de3edf0f56aa9716b898ec8'
ETHERMINE_WALLET
def query_normal_transactions(wallet_address, startblock=0, endblock=99999999): 
  """
    Ping Etherscan.io and get the most recent 10k transactions for this wallet.
    Each of these takes ~4 seconds
  """
  api_request_text =f'https://api.etherscan.io/api?module=account&action=txlist&address={wallet_address}&startblock={startblock}&endblock={endblock}&sort=asc&apikey={ETHERSCAN_API_KEY}'
     
  response = requests.get(api_request_text) # you cannot make more than 5 requests per second. This has yet to be a problem
  api_response_text = json.loads(response.text) 
  
  # error handling to make sure that the call is going through
  if api_response_text['message'] != 'OK': # untested
    print(f'you got an error at address={wallet_address} \nstartblock= {startblock}\nendblock={endblock}')

  else:
    transaction_list = api_response_text['result']
    simplified_transactions = parse_normal_transactions(transaction_list=transaction_list, wallet_address=wallet_address)

    #exclude the last N transactions since the might overlapp
    largest_block = int(simplified_transactions[-1][2]) # O(1)
    while simplified_transactions[-1][2] == largest_block:
      simplified_transactions.pop(-1) # O(1) * 100 approx upper bound. This gets rid of all the blocks that are the same as the last block

    column_names = ['to_address',
                    'from_address',
                    'block_number',
                    'transaction_year',
                    'transaction_month',
                    'value_in_ether']
    
    # create a rdd of this batch of transactions
    rdd = spark.createDataFrame(simplified_transactions, schema=column_names)
    return rdd, largest_block

def parse_normal_transactions(transaction_list: list, wallet_address: str)-> list:
    """
      Converts transaction data into a easy to read from and excludes the last block
    :param transaction_list: A list of dictionary objects. Each dictionary is a single transaction.
    :return:
        simple_transactions: a list of tuples storing (to_address, from_address, Block number, Year, Month, Value in Ether)
                                      Data types are: (String, String, int, int, int, float)
    """
    simplified_transactions =[(trans_dict['to'],
                              trans_dict['from'],
                              int(trans_dict['blockNumber']),
                              datetime.datetime.fromtimestamp(int(trans_dict['timeStamp'])).year,
                               datetime.datetime.fromtimestamp(int(trans_dict['timeStamp'])).month,
                              int(trans_dict['value'])/math.pow(10, 18)) # convert the 'value' column from wei to Ether
                              for trans_dict in transaction_list if trans_dict['from'] == wallet_address
                              ]
    return simplified_transactions

## Next steps are to get a dictionary of block: time chunks. 




In [16]:
start_year = 2016
years = [datetime.datetime(start_year+i,1,1) for i in range(0,6)]
unix_timestamp_years = [int(year.timestamp()) for year in years]
unix_timestamp_march31_2016 = int(datetime.datetime(2021,3,31).timestamp())

command_for_block_after_timestamp = f'https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={unix_timestamp_years[0]}&closest=after&apikey={ETHERSCAN_API_KEY}'

year_block_dict ={}

for unix_time in unix_timestamp_years:
  time.sleep(.2) # don't overwhelm the API
  response = requests.get(f'https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={unix_time}&closest=after&apikey={ETHERSCAN_API_KEY}')
  response_text = json.loads(response.text) 
  block = response_text['result']
  year_block_dict[int(datetime.datetime.fromtimestamp(unix_time).year)]= block

print(year_block_dict)
response = requests.get(f'https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={unix_timestamp_march31_2016}&closest=after&apikey={ETHERSCAN_API_KEY}')
response_text = json.loads(response.text) 
march_2021_block = response_text['result']
march_2021_block
year_block_dict['2022'] = march_2021_block # this is to make the code work. as written 

{2016: '778483', 2017: '2912407', 2018: '4832686', 2019: '6988615', 2020: '9193266', 2021: '11565019'}


In [3]:
def get_monthly_miner_income_statements_chunk(year,pool_address):
  """
    Ping every transaction where {pool_address} is the sender and cast it as a pyspark dataframe.

    add a year_month column. 

    Group by to_address and year_month

    cast as a pandas df

    Save the document to a file in this local runtime of google drive. 
  """
  start = datetime.datetime.now()
  min_block = int(year_block_dict[year]) # if year = 2018 then this gets all miner income in 2018
  max_block = int(year_block_dict[year+1]) # all blocks must be within this range


  i=0
  #Inital call to set up transaction rdd. You will union transactions on to this.
  transactions_rdd, largest_block = query_normal_transactions(wallet_address=pool_address, startblock=min_block, endblock=max_block)

  while largest_block < max_block:
    try:
      # repeat the call picking up where you left off last call since you can only get 10,000 records per call
      next_chunk_rdd, largest_block = query_normal_transactions(wallet_address=pool_address, startblock=largest_block) 
      transactions_rdd = transactions_rdd.union(next_chunk_rdd) # union removes duplicates
      # this lets you see progress
      if i %10 ==0:
        time_dif = datetime.datetime.now() - start
        start = datetime.datetime.now()
        print(f'Call:{i} Time:{time_dif} Block:{largest_block} Blocks Left:{max_block - largest_block}')
      i+=1
    except:
      print(f'you got an error at {largest_block}\n {next_chunk_rdd.head(1)} ')

  
  # add the year_month column. 
  transactions_rdd = transactions_rdd.withColumn('year_month', concat(transactions_rdd.transaction_year,lit("-"),
                                                                      transactions_rdd.transaction_month))
  global miner_df
  miner_df = group_miners(transactions_rdd,pool_address, year)
  save_miner_df_to_drive(miner_df, year, pool_address)



def group_miners(transactions_rdd, pool_address, year):
  """

    Group the transaction rdd into year_month, to address groups by summing the value_in_ether. 
  input: the transaction rdd for a single year.

  Do this of this sql command

     SELECT to_address, year_month, ROUND(sum(value_in_ether),9) as eth_earned
                                    FROM raw_transactions 
                                    Where block_number < {max_block} and block_number > {min_block}
                                    GROUP BY to_address, year_month
                                    ORDER BY to_address, year_month  
  """

  miner_rdd = transactions_rdd.groupby('to_address','year_month').sum('value_in_ether') 
  miner_rdd = miner_rdd.withColumn('from_address', lit(pool_address)) # add the pool address as a column might be unneccessary since you have this in the header
  miner_rdd = miner_rdd.filter(miner_rdd['year_month'].contains(str(year))) # only include the  from the year you are considering
  print('You have done data aggregation')
  df = miner_rdd.toPandas()

  # you might want to drop the other columns
  # you would drop (Year, Month, and pool address column since it is in the header.
  print('Verify that these months are correct')
  print(df['year_month'].unique())
  return df

def save_miner_df_to_drive(miner_df, year, pool_address):
  """
    Save the miner_df to your google drive in a particular folder
    df: a grouped miner income statement
    yera: the year you are saving data for. 
    pool_address: the ETH wallet address of the pool
  """
  PATH_TO_SAVE = f'/content/drive/MyDrive/transaction_data/miner_income_{year}_{pool_address}.csv'
  miner_df.to_csv(PATH_TO_SAVE,index=False) # save to your google drive
  print('Saved to google drive')




In [5]:
HIVEON_POOL= '0x1aD91ee08f21bE3dE0BA2ba6918E714dA6B45836'
SPARK_POOL = '0x5A0b54D5dc17e0AadC383d2db43B0a0D3E029c4c'
F2_POOL = '0x829BD824B016326A401d083B33D092293333A830'
ZHIZHU_TOP_POOL = '0x04668Ec2f57cC15c381b461B9fEDaB5D451c8F7F'
ETHERMINE_POOL = '0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8'

POOLS = [ETHERMINE_POOL, HIVEON_POOL, SPARK_POOL, F2_POOL, ZHIZHU_TOP_POOL]

In [6]:
%%time
years = [2016,2017,2018,2019,2020,2021]
for year in years:
  for pool in POOLS:
    get_monthly_miner_income_statements_chunk(year,pool)

    #I think this should just work now when I run it from the top

NameError: ignored

In [ ]:
# miner_rdd = local_transaction_rdd.groupby('to_address','year_month').sum('value_in_ether') 
# miner_rdd.toPandas()['year_month'].unique()



In [ ]:
# miner_rdd = local_transaction_rdd.groupby('to_address','year_month').sum('value_in_ether') 
# miner_rdd = miner_rdd.withColumn('from_address', lit(pool_address)) # add the pool address as a column might be unneccessary since you have this in the header
# miner_rdd = miner_rdd.filter(miner_rdd['year_month'].contains(str(year))) # only include the data where the month_year.conaints '2016
# df = miner_rdd.toPandas()
# # you might want to drop the other columns
# # you would drop (Year, Month, and )
# print('Verify that these months are correct')
# print(df['year_month'].unique())

### Estimated Timecosts 

This is the number of API calls you need to make for Ethermine.io 
Spark pool is similar in size, the other pools are much smaller. 

In [ ]:
# How many API calls do you need to make?
n_transactions = 40000000
reccords_per_call = 10000
total_api_calls = n_transactions/reccords_per_call
base_time_cost_per_api_call = 5 #Loose Uppperbound It is really 3.5 seconds
seconds_needed = (base_time_cost_per_api_call *total_api_calls) 
hours = seconds_needed/3600 # 3600 is seconds in an hour
print(f'Upper bound for API calls for ethermine {total_api_calls_needed}') # upper bound is 5 hours.
print(f'Upper bound for hours to get all of Ethermine hours to get everything from ethermine {hours}')

In [ ]:


# def group_miners1(transactions_rdd, max_block, min_block):
#   """
#   Old version works at small scale breaks  as you hit year 3  due to memeory leak somewhere. 
#   If you restart the run every year it ought to work
#   """
#   transactions_rdd.createTempTable("raw_transactions") 

#   monthly_miner_revenue_query =f""" SELECT to_address, year_month, ROUND(sum(value_in_ether),9) as eth_earned
#                                     FROM raw_transactions 
#                                     Where block_number < {max_block} and block_number > {min_block}
#                                     GROUP BY to_address, year_month
#                                     ORDER BY to_address, year_month 
#                                 """

#   miners_sql_df = spark.sql(monthly_miner_revenue_query)
#   miners_sql_df = miners_sql_df.withColumn('from_address', lit(pool_address)) # add the pool address back into the rdd
#   df = miners_sql_df.toPandas()
#   print('Verify that these months are correct')
#   print(df['year_month'].unique())
#   return df